In [1]:
import os, json
import pandas as pd

In [2]:
years = [year for year in range(2016,2022+1)]
dfs_dfs = []
for year in years:
    path_str = 'data/DFS_Salaries/' + str(year) + '/'
    for x, file_name in enumerate([x_json for x_json in os.listdir(path_str) if x_json.endswith('.json')]):
        with open(path_str + file_name) as json_file:
            if x == 0:
                dfs_df = pd.DataFrame(json.load(json_file))
            else:
                dfs_df = pd.concat([dfs_df,pd.DataFrame(json.load(json_file))])
    dfs_df['Season'] = year
    dfs_dfs.append(dfs_df)

In [3]:
dfs_dfs[6]

,Name,Team,Position,Week,Opponent,FantasyPointsDraftKings,UpcomingOpponentRank,UpcomingOpponentPositionRank,DraftKingsSalary,Rank,Season
0,Tampa Bay Buccaneers,TB,DST,7,CAR,12.7,None,None,3900.0,1,2022
1,New England Patriots,NE,DST,7,CHI,10.3,None,None,3800.0,2,2022
2,Pittsburgh Steelers,PIT,DST,4,NYJ,10.3,None,None,3600.0,2,2022
3,Dallas Cowboys,DAL,DST,8,CHI,10.0,None,None,4000.0,4,2022
4,Minnesota Vikings,MIN,DST,5,CHI,9.8,None,None,3600.0,5,2022
...,...,...,...,...,...,...,...,...,...,...,...
1997,Garrett Gilbert,NE,QB,7,CHI,0.0,9.0,11.0,4800.0,1207,2022
1998,Andy Isabella,BAL,WR,7,CLE,0.0,24.0,23.0,3000.0,1207,2022
1999,Nick Eubanks,HOU,TE,7,LV,0.0,25.0,30.0,NaN,1207,2022
2000,Keelan Doss,LAC,WR,7,SEA,0.0,29.0,7.0,3000.0,1207,2022


In [4]:
#combine all years of dataframes
dfs_df = pd.concat(dfs_dfs)

In [5]:
#create a unique id for grouping
dfs_df['grouping'] = dfs_df['Team']+dfs_df['Position']+dfs_df['Season'].astype(str)+dfs_df['Week'].astype(str).str.zfill(2)

In [6]:
#inverting salary to groupby "first" which relies on minimum instead of maximum
dfs_df['inv_sal'] = 1/dfs_df['DraftKingsSalary']

In [7]:
#get ranking of minimum inverse salaries by team/position/week/season which is the same as ranking of descending max salaries
dfs_df['pos_team_rank_week'] = dfs_df.groupby('grouping')['inv_sal'].rank(method = 'first')

In [8]:
#create a new column to ensure no duplicates occured when ranking
dfs_df['dup_test'] = dfs_df['grouping'] + dfs_df['pos_team_rank_week'].astype(str)

In [9]:
dfs_df.to_csv('test_sals.csv')

In [10]:
#ensuring all non-NaN values are not duplicated
dfs_df[dfs_df['DraftKingsSalary'].notna()].duplicated(subset = ['dup_test']).unique()

array([False])

In [11]:
dfs_df['pos_rank'] = dfs_df['Position'] + dfs_df['pos_team_rank_week'].astype(str)

In [12]:
#make new dataframe for testing
dfs_df_test = dfs_df.dropna(axis = 0, subset = ['DraftKingsSalary']).reset_index(drop=True)

In [13]:
dfs_df_test

,Name,Team,Position,Week,Opponent,FantasyPointsDraftKings,UpcomingOpponentRank,UpcomingOpponentPositionRank,DraftKingsSalary,Rank,Season,grouping,inv_sal,pos_team_rank_week,dup_test,pos_rank
0,Seattle Seahawks,SEA,DST,15,LAR,11.9,None,None,4000.0,1,2016,SEADST201615,0.000250,1.0,SEADST2016151.0,DST1.0
1,Seattle Seahawks,SEA,DST,2,LAR,11.5,None,None,3800.0,2,2016,SEADST201602,0.000263,1.0,SEADST2016021.0,DST1.0
2,Philadelphia Eagles,PHI,DST,9,NYG,10.6,None,None,3100.0,3,2016,PHIDST201609,0.000323,1.0,PHIDST2016091.0,DST1.0
3,Seattle Seahawks,SEA,DST,1,MIA,10.5,None,None,3900.0,4,2016,SEADST201601,0.000256,1.0,SEADST2016011.0,DST1.0
4,Arizona Cardinals,ARI,DST,6,NYJ,10.5,None,None,3800.0,4,2016,ARIDST201606,0.000263,1.0,ARIDST2016061.0,DST1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70085,Davion Davis,HOU,WR,7,LV,0.0,28.0,21.0,3000.0,1207,2022,HOUWR202207,0.000333,10.0,HOUWR20220710.0,WR10.0
70086,Garrett Gilbert,NE,QB,7,CHI,0.0,9.0,11.0,4800.0,1207,2022,NEQB202207,0.000208,3.0,NEQB2022073.0,QB3.0
70087,Andy Isabella,BAL,WR,7,CLE,0.0,24.0,23.0,3000.0,1207,2022,BALWR202207,0.000333,9.0,BALWR2022079.0,WR9.0
70088,Keelan Doss,LAC,WR,7,SEA,0.0,29.0,7.0,3000.0,1207,2022,LACWR202207,0.000333,10.0,LACWR20220710.0,WR10.0


In [14]:
#cast as int then str to remove decimal
dfs_df_test['pos_team_rank_week'] = dfs_df_test['pos_team_rank_week'].astype(int).astype(str)

In [15]:
dfs_df_test['pos_rank'] = dfs_df_test['Position'] + dfs_df_test['pos_team_rank_week']

In [16]:
dfs_df_test

,Name,Team,Position,Week,Opponent,FantasyPointsDraftKings,UpcomingOpponentRank,UpcomingOpponentPositionRank,DraftKingsSalary,Rank,Season,grouping,inv_sal,pos_team_rank_week,dup_test,pos_rank
0,Seattle Seahawks,SEA,DST,15,LAR,11.9,None,None,4000.0,1,2016,SEADST201615,0.000250,1,SEADST2016151.0,DST1
1,Seattle Seahawks,SEA,DST,2,LAR,11.5,None,None,3800.0,2,2016,SEADST201602,0.000263,1,SEADST2016021.0,DST1
2,Philadelphia Eagles,PHI,DST,9,NYG,10.6,None,None,3100.0,3,2016,PHIDST201609,0.000323,1,PHIDST2016091.0,DST1
3,Seattle Seahawks,SEA,DST,1,MIA,10.5,None,None,3900.0,4,2016,SEADST201601,0.000256,1,SEADST2016011.0,DST1
4,Arizona Cardinals,ARI,DST,6,NYJ,10.5,None,None,3800.0,4,2016,ARIDST201606,0.000263,1,ARIDST2016061.0,DST1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70085,Davion Davis,HOU,WR,7,LV,0.0,28.0,21.0,3000.0,1207,2022,HOUWR202207,0.000333,10,HOUWR20220710.0,WR10
70086,Garrett Gilbert,NE,QB,7,CHI,0.0,9.0,11.0,4800.0,1207,2022,NEQB202207,0.000208,3,NEQB2022073.0,QB3
70087,Andy Isabella,BAL,WR,7,CLE,0.0,24.0,23.0,3000.0,1207,2022,BALWR202207,0.000333,9,BALWR2022079.0,WR9
70088,Keelan Doss,LAC,WR,7,SEA,0.0,29.0,7.0,3000.0,1207,2022,LACWR202207,0.000333,10,LACWR20220710.0,WR10


In [17]:
dfs_df2 = dfs_df_test.pivot_table(index = ['Team','Season','Week','Opponent'], columns = 'pos_rank',values = 'DraftKingsSalary',aggfunc = 'sum').reset_index()

In [18]:
dfs_df_final = dfs_df2[['Team','Season','Week','Opponent','DST1','QB1','RB1','RB2','WR1','WR2','WR3','WR4','TE1','TE2']]

In [19]:
dfs_df_final.to_csv('data/dfs_salaries.csv')

In [20]:
dfs_df_final['Team'].unique()

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LAR', 'LV',
       'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype=object)